In [9]:
import tkinter as tk
from tkinter import messagebox


class Grilles:
    def __init__(self, grille_affichage, grille_modifiable):
        self.grille_affichage = grille_affichage
        self.grille_modifiable = grille_modifiable

def afficher_mot_croise_modifiable(grilles):
    # Créer la fenêtre principale
    root = tk.Tk()
    root.title("Mot Croisé")

    # Dictionnaires pour stocker les widgets
    entries_modifiable = {}
    labels_affichage = {}

    # Créer des Frames pour les grilles
    frame_modifiable = tk.Frame(root)
    frame_modifiable.grid(row=1, column=0, padx=10, pady=10)

    frame_affichage = tk.Frame(root)
    frame_affichage.grid(row=1, column=1, padx=10, pady=10)

    # Fonction pour mettre à jour la grille modifiable avec les nouvelles lettres
    def mettre_a_jour_grille():
        for (i, j), entry in entries_modifiable.items():
            nouvelle_lettre = entry.get().upper()
            if entry.cget('bg') == '#404040':  # Case noire foncée
                grilles.grille_modifiable[i][j] = 1
            else:
                grilles.grille_modifiable[i][j] = nouvelle_lettre if nouvelle_lettre != '' else 0
        print("Nouvelle grille modifiable mise à jour :", grilles.grille_modifiable)
        mettre_a_jour_affichage()

    # Fonction pour mettre à jour la grille d'affichage
    def mettre_a_jour_affichage():
        for (i, j), label in labels_affichage.items():
            valeur = grilles.grille_affichage[i][j]
            if valeur == 1:
                label.config(bg="#404040", fg="white", text="")
            elif valeur == 0:
                label.config(bg="white", fg="black", text="")
            else:
                label.config(bg="white", fg="black", text=valeur.upper())

    # Fonction pour forcer les majuscules lors de la saisie
    def forcer_majuscules(event):
        if event.widget.cget('bg') != '#404040':  # Ne pas forcer les majuscules si la case est noire
            event.widget.delete(0, tk.END)
            event.widget.insert(0, event.char.upper())

    # Fonction pour gérer le clic droit et transformer une case en noire ou en vide
    def transformer_case(event, i, j):
        entry = entries_modifiable[(i, j)]
        if entry.cget('bg') == '#404040':  # Si déjà noire foncée, la rendre vide
            entry.config(bg="white", fg="black", state="normal")
            entry.delete(0, tk.END)  # Supprimer le texte s'il y en a
            grilles.grille_modifiable[i][j] = 0
        else:  # Si pas noire, la rendre noire foncée
            entry.config(bg="#404040", fg="white", state="normal")
            entry.delete(0, tk.END)  # Supprimer le texte pour ne garder que le noir
            grilles.grille_modifiable[i][j] = 1

    # Fonction pour mettre à jour les grilles en fonction des dimensions fournies
    def appliquer_dimensions():
        try:
            lignes = int(entry_lignes.get())
            colonnes = int(entry_colonnes.get())
        except ValueError:
            messagebox.showerror("Erreur", "Les valeurs de lignes et colonnes doivent être des entiers.")
            return

        # Mise à jour des grilles
        grilles.grille_modifiable = [[0 for _ in range(colonnes)] for _ in range(lignes)]
        grilles.grille_affichage = [[0 for _ in range(colonnes)] for _ in range(lignes)]

        # Réinitialiser les Frames
        for widget in frame_modifiable.winfo_children():
            widget.destroy()
        for widget in frame_affichage.winfo_children():
            widget.destroy()

        # Recréer les grilles avec les nouvelles dimensions
        creer_grille_modifiable()
        creer_grille_affichage()

    # Fonction pour créer la grille modifiable
    def creer_grille_modifiable():
        for i, ligne in enumerate(grilles.grille_modifiable):
            for j, lettre in enumerate(ligne):
                entry = tk.Entry(frame_modifiable, font=("Helvetica", 24), width=3, justify='center', borderwidth=2, relief="solid")
                entry.grid(row=i, column=j, padx=5, pady=5)
                
                # Si la case est noire (définie comme 1), alors la colorer en noir foncé
                if lettre == 1:
                    entry.config(bg="#404040", fg="white", state="normal")  # Normal pour permettre le clic droit
                elif lettre != 0:  # Comparer avec 0 pour remplir la case si elle n'est pas vide
                    entry.insert(0, lettre)
                    entry.config(fg="black")
                
                # Lier l'événement KeyRelease à la fonction forcer_majuscules pour forcer la saisie en majuscule, sauf si la case est noire
                entry.bind("<KeyRelease>", forcer_majuscules)
                
                # Lier l'événement clic droit pour transformer la case en noire ou en vide
                entry.bind("<Button-3>", lambda event, i=i, j=j: transformer_case(event, i, j))
                
                # Sauvegarder l'entry dans le dictionnaire avec sa position comme clé
                entries_modifiable[(i, j)] = entry

    # Fonction pour créer la grille d'affichage
    def creer_grille_affichage():
        print(grilles.grille_affichage)
        for i, ligne in enumerate(grilles.grille_affichage):
            for j, lettre in enumerate(ligne):
                label = tk.Label(frame_affichage, font=("Helvetica", 24), width=3, justify='center', borderwidth=2, relief="solid")
                label.grid(row=i, column=j, padx=5, pady=5)

                # Si la case est noire (définie comme 1), alors la colorer en noir foncé
                if lettre == 1:
                    label.config(bg="#404040", fg="white")
                elif lettre == 0:  # Comparer avec 0 pour laisser la case vide
                    label.config(bg="white", fg="black")
                else:
                    label.config(text=lettre.upper(), bg="white", fg="black")  # Assurer que les lettres sont en majuscule

                # Sauvegarder le label dans le dictionnaire avec sa position comme clé
                labels_affichage[(i, j)] = label

    # Zone pour entrer le nombre de lignes et de colonnes
    tk.Label(root, text="Nombre de lignes:").grid(row=0, column=0, padx=10, pady=10, sticky="e")
    entry_lignes = tk.Entry(root)
    entry_lignes.grid(row=0, column=1, padx=10, pady=10)

    tk.Label(root, text="Nombre de colonnes:").grid(row=0, column=2, padx=10, pady=10, sticky="e")
    entry_colonnes = tk.Entry(root)
    entry_colonnes.grid(row=0, column=3, padx=10, pady=10)

    # Bouton pour appliquer les dimensions
    bouton_appliquer = tk.Button(root, text="Appliquer Dimensions", command=appliquer_dimensions)
    bouton_appliquer.grid(row=0, column=4, padx=10, pady=10)

    # Bouton pour mettre à jour la grille modifiable
    bouton_mise_a_jour = tk.Button(root, text="Mettre à jour la grille", command=mettre_a_jour_grille)
    bouton_mise_a_jour.grid(row=2, columnspan=5, pady=10)

    # Créer les grilles initiales avec les dimensions par défaut
    creer_grille_modifiable()
    creer_grille_affichage()

    # Lancer la boucle principale de l'interface
    root.mainloop()

# Exemple initial de grille modifiable et grille d'affichage
grille_modifiable_test = [
    ['P', 'Y', 'T', 'H', 'O', 'N'],
    [0, 1, 0, 'O', 0, 0],
    [0, 0, 0, 'D', 'E', 'V'],
    [0, 'C', 'O', 'D', 'E', 1],
]

grille_affichage_test = [
    ['A', 'B', 'C', 'D', 'E', 'F'],
    [0, 0, 1, 'G', 0, 1],
    [1, 0, 1, 'H', 'I', 'J'],
    [0, 'K', 'L', 'M', 0, 0],
]

grilles = Grilles(grille_affichage_test, grille_modifiable_test)
# Appeler la fonction pour afficher les deux grilles
afficher_mot_croise_modifiable(grilles)

[['A', 'B', 'C', 'D', 'E', 'F'], [0, 0, 1, 'G', 0, 1], [1, 0, 1, 'H', 'I', 'J'], [0, 'K', 'L', 'M', 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Nouvelle grille modifiable mise à jour : [[1, 0, 1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


: 